## Generate Morpheus Input Data

**Gregory Way, 2019**

Use this script to concatenate all of the cell painting data into one `.gct` file for input into morpheus.

In [1]:
import os
import pandas as pd

In [2]:
%load_ext rpy2.ipython

In [3]:
def load_files(plate_dir_list, file_grep="normalized_variable_selected.csv"):
    # Build full cell painting dataset
    df_list = []
    for plate_dir in plate_dirs:
        if ".DS_Store" in plate_dir:
            continue
        plate_files = os.listdir(plate_dir)
        for plate_file in plate_files:
            if file_grep in plate_file:
                plate_file = os.path.join(plate_dir, plate_file)
                df = pd.read_csv(plate_file)
                print("reading {} with profile count: {}".format(plate_file, df.shape[0]))
                df_list.append(df)
                
    cp_df = pd.concat(df_list).reset_index(drop=True)
    return cp_df

## Batch 1

In [4]:
batch_id = "2019_04_16_Batch1"
backend_dir = os.path.join("..", "..", "backend", batch_id)

plate_dirs = [os.path.join(backend_dir, x) for x in os.listdir(backend_dir)]

In [5]:
cp_df = load_files(plate_dirs, file_grep="normalized_variable_selected.csv")
cp_df.Metadata_diff_day = cp_df.Metadata_diff_day.astype(str)

print(cp_df.shape)
cp_df.head()

reading ../../backend/2019_04_16_Batch1/BR00101080/BR00101080_normalized_variable_selected.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101081/BR00101081_normalized_variable_selected.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101075/BR00101075_normalized_variable_selected.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101079/BR00101079_normalized_variable_selected.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101077/BR00101077_normalized_variable_selected.csv with profile count: 56
reading ../../backend/2019_04_16_Batch1/BR00101083/BR00101083_normalized_variable_selected.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101082/BR00101082_normalized_variable_selected.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101076/BR00101076_normalized_variable_selected.csv with profile count: 54
reading ../../backend/2019_04_16_Batch1/BR00101078/BR001

,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_cell_line,Metadata_patient,Metadata_FFA,Metadata_diff_day,Cells_AreaShape_EulerNumber,...,Nuclei_Texture_InfoMeas2_DNA_10_01,Nuclei_Texture_InfoMeas2_Mito_10_03,Nuclei_Texture_InverseDifferenceMoment_DNA_10_00,Nuclei_Texture_InverseDifferenceMoment_ER_10_02,Nuclei_Texture_InverseDifferenceMoment_Mito_10_02,Nuclei_Texture_InverseDifferenceMoment_Mito_20_02,Nuclei_Texture_SumAverage_DNA_20_03,Nuclei_Texture_SumVariance_AGP_20_01,Nuclei_Texture_SumVariance_ER_20_03,Nuclei_Texture_SumVariance_Mito_20_01
0,BR00101080,B02,BR00101080,BR00101080,B02,vc,PAC_261,0,3,-3.840565,...,-1.814082,0.039732,1.401266,-0.336906,-0.388075,2.894933,0.674111,0.915279,0.533406,0.954802
1,BR00101080,B03,BR00101080,BR00101080,B03,sc,PAC_246,0,3,-0.178843,...,-1.324510,-0.330499,1.344428,2.740432,0.738743,0.208088,-0.359262,-0.526420,-0.707300,-0.273817
2,BR00101080,B04,BR00101080,BR00101080,B04,vc,PAC_246,0,3,0.627837,...,-0.281201,0.867523,0.164746,-0.332073,-0.595390,-0.495383,-0.599370,-0.536304,-0.745026,-0.146896
3,BR00101080,B05,BR00101080,BR00101080,B05,sc,PAC_266,0,3,-0.134838,...,-0.480267,0.412462,0.433084,0.885338,-0.114261,-0.603112,-0.771305,-0.499049,-0.822919,-0.105638
4,BR00101080,B06,BR00101080,BR00101080,B06,vc,PAC_266,0,3,-3.069128,...,-2.483936,-1.712249,2.193710,1.653152,1.362144,6.744472,1.021106,0.937149,0.186470,0.288610


In [6]:
# Extract out the cell painting features selected in this batch
batch_1_features = [x for x in cp_df.columns if not x.startswith("Metadata_")]

In [7]:
# Output combined file
file = os.path.join("data", "combined_normalized_variable_selected.tsv")
cp_df.to_csv(file, index=False, sep='\t')

# Collapse Data for Morpheus Heatmaps

In [8]:
# Extract out day 15 data
cp_15_df = cp_df.query("Metadata_diff_day in ['15', '15+iso']").reset_index(drop=True)
print(cp_15_df.shape)

cp_non15_df = cp_df.query("Metadata_diff_day not in ['15', '15+iso']").reset_index(drop=True)
print(cp_non15_df.shape)

(114, 549)
(416, 549)


In [9]:
replicate_cols = ["Metadata_Plate",
                  "Metadata_cell_line",
                  "Metadata_patient",
                  "Metadata_FFA",
                  "Metadata_diff_day"]

cp_15_collapsed_df = cp_15_df.groupby(replicate_cols).mean().reset_index()
cp_non15_collapsed_df = cp_non15_df.groupby(replicate_cols).mean().reset_index()

## Use `write_gct.R` to build the Moxrpheus Input

In [10]:
%%R -i cp_df -i batch_id -i backend_dir -i cp_15_df -i cp_non15_df -i cp_15_collapsed_df -i cp_non15_collapsed_df

library(dplyr)
library(magrittr)

file <- file.path("..", "cytominer_scripts", "write_gct.R")
source(file)

output <- file.path("results", "morpheus",
                    paste0("full_", batch_id, "_morpheus.gct"))
channels <- NULL
create_row_annotations <- TRUE
feature_regex <- "^Nuclei_|^Cells_|^Cytoplasm_"

# Step 1: Output combined gct file
write_gct(x = cp_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

# Step 2: Output specific plate combinations
# (replicate collapsed and non replicate collapsed)
output <- file.path("results", "morpheus",
                    paste0("full_", batch_id, "_morpheus_day15.gct"))

write_gct(x = cp_15_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

output <- file.path("results", "morpheus",
                    paste0("full_", batch_id, "_morpheus_nonday15.gct"))

write_gct(x = cp_non15_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

output <- file.path("results", "morpheus",
                    paste0("full_", batch_id, "_morpheus_day15_collapsed.gct"))

write_gct(x = cp_15_collapsed_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

output <- file.path("results", "morpheus",
                    paste0("full_", batch_id, "_morpheus_nonday15_collapsed.gct"))

write_gct(x = cp_non15_collapsed_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

# Step 3: Also generate and write individual gct files
plate_cols <- readr::cols(
    .default = readr::col_double(),
    Metadata_Plate = readr::col_character(),
    Metadata_Well = readr::col_character(),
    Metadata_Assay_Plate_Barcode = readr::col_character(),
    Metadata_Plate_Map_Name = readr::col_character(),
    Metadata_well_position = readr::col_character(),
    Metadata_cell_line = readr::col_character(),
    Metadata_patient = readr::col_character(),
    Metadata_FFA = readr::col_character(),
    Metadata_diff_day = readr::col_character()
)

all_plate_dirs <- list.files(backend_dir, full.names = TRUE)
for (plate_dir in all_plate_dirs) {
    plate_file <- list.files(plate_dir, full.names = FALSE, pattern = "normalized_variable_selected")[1]
    full_plate_file <- file.path(plate_dir, plate_file)

    df <- readr::read_csv(full_plate_file, col_types = plate_cols)
    
    output_file <- file.path("results", "morpheus",
                             paste0(tools::file_path_sans_ext(plate_file),
                                    "_", batch_id, "_morpheus.gct"))
    write_gct(x = df,
              path = output_file,
              channels = channels,
              create_row_annotations = create_row_annotations,
              feature_regex = feature_regex)
}

/Users/gway/miniconda3/envs/t2d-v2f/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)
/Users/gway/miniconda3/envs/t2d-v2f/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/Users/gway/miniconda3/envs/t2d-v2f/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/Users/gway/miniconda3/envs/t2d-v2f/lib/python3.7/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)


## Batch 2

In [11]:
batch_id = "2019_06_11_Batch2"
file = os.path.join("data", "merged_profiles_{}.tsv.gz".format(batch_id))

cp_df = pd.read_csv(file, sep='\t')
print(cp_df.shape)
cp_df.head(3)

(312, 453)


,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_cell_line,Metadata_condition_O2,Metadata_treatment,Cells_AreaShape_Compactness,Cells_AreaShape_EulerNumber,...,Nuclei_Texture_SumAverage_AGP_20_02,Nuclei_Texture_SumAverage_ER_20_00,Nuclei_Texture_SumAverage_Mito_20_01,Nuclei_Texture_SumEntropy_DNA_5_02,Nuclei_Texture_SumVariance_AGP_10_03,Nuclei_Texture_SumVariance_DNA_10_01,Nuclei_Texture_SumVariance_DNA_20_00,Nuclei_Texture_SumVariance_DNA_20_03,Nuclei_Texture_SumVariance_Mito_20_03,Nuclei_Texture_SumVariance_Mito_5_02
0,BR00101066,B02,BR00101066,BR00101066,B02,SNU761,21,OA,0.851890,-6.002905,...,-0.354477,0.325898,-1.209791,-1.002227,0.304018,-0.441063,-1.493966,-1.256423,-0.875879,-0.896676
1,BR00101066,B03,BR00101066,BR00101066,B03,SNU761,21,PA,-0.526301,-0.216030,...,-0.610291,-0.521134,-0.660242,1.687832,0.867847,1.368041,0.453843,-0.515845,0.042559,0.695480
2,BR00101066,B04,BR00101066,BR00101066,B04,SNU761,21,OA_PA,-0.598634,-0.071529,...,2.984472,1.634340,4.691127,-1.523420,3.588932,-1.171869,0.616111,1.229639,1.907388,0.133260


In [12]:
# Merge replicate cols
replicate_cols = ["Metadata_Plate",
                  "Metadata_cell_line",
                  "Metadata_condition_O2",
                  "Metadata_treatment"]

cp_collapse_df = cp_df.groupby(replicate_cols).mean().reset_index()

## Use `write_gct.R` to build the Morpheus Input

In [13]:
%%R -i cp_df -i batch_id -i cp_collapse_df

library(dplyr)
library(magrittr)

file <- file.path("..", "cytominer_scripts", "write_gct.R")
source(file)

output <- file.path("results", "morpheus",
                    paste0("full_", batch_id, "_morpheus.gct"))
output_collapsed <- file.path("results", "morpheus",
                              paste0("collapsed_", batch_id, "_morpheus.gct"))

channels <- NULL
create_row_annotations <- TRUE
feature_regex <- "^Nuclei_|^Cells_|^Cytoplasm_"

# Step 1: Output combined gct file
write_gct(x = cp_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

# Step 2: Output replicate collapsed gct file
write_gct(x = cp_collapse_df,
          path = output_collapsed,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

## Batch 3

In [14]:
batch_id = "2019_08_06_Batch3"
backend_dir = os.path.join("..", "..", "backend", batch_id)

plate_dirs = [os.path.join(backend_dir, x) for x in os.listdir(backend_dir)]

In [15]:
cp_df = load_files(plate_dirs, file_grep="normalized_variable_selected.csv")
cp_df.Metadata_diff_day = cp_df.Metadata_diff_day.astype(str)
cp_df.Metadata_patient = cp_df.Metadata_patient.astype(str)

print(cp_df.shape)
cp_df.head()

reading ../../backend/2019_08_06_Batch3/D14/D14_normalized_variable_selected.csv with profile count: 80
reading ../../backend/2019_08_06_Batch3/Day3/Day3_normalized_variable_selected.csv with profile count: 80
reading ../../backend/2019_08_06_Batch3/Day0/Day0_normalized_variable_selected.csv with profile count: 80
reading ../../backend/2019_08_06_Batch3/Day8/Day8_normalized_variable_selected.csv with profile count: 80
(320, 381)


,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_cell_line,Metadata_patient,Metadata_FFA,Metadata_diff_day,Cells_AreaShape_Eccentricity,...,Nuclei_Texture_InfoMeas1_ER_5_02,Nuclei_Texture_InfoMeas1_ER_5_03,Nuclei_Texture_InfoMeas1_Mito_5_02,Nuclei_Texture_InfoMeas2_AGP_5_00,Nuclei_Texture_InfoMeas2_DNA_5_02,Nuclei_Texture_InfoMeas2_ER_5_00,Nuclei_Texture_SumAverage_DNA_20_03,Nuclei_Texture_SumEntropy_DNA_20_01,Nuclei_Texture_SumVariance_AGP_5_02,Nuclei_Texture_SumVariance_DNA_5_01
0,D14,A01,D14,D14,A01,sc,252,0,14,0.630662,...,-2.125743,-2.665606,-2.356083,0.682714,-0.725557,1.865769,-2.826216,-3.226772,-0.090367,0.149223
1,D14,A02,D14,D14,A02,vc,252,0,14,0.314300,...,-0.504041,-0.845735,0.011658,0.135625,-0.753222,0.709504,0.677852,1.411267,0.193334,-0.612095
2,D14,A04,D14,D14,A04,sc,214,0,14,1.060115,...,-0.505854,-0.615315,-0.732776,0.090121,0.718639,0.633794,0.975302,1.887109,-0.063542,0.677937
3,D14,A05,D14,D14,A05,vc,214,0,14,0.067539,...,-0.085408,-0.234523,0.018579,-0.547645,-0.538573,-0.179935,-0.275668,0.295252,-0.033855,-0.710885
4,D14,A06,D14,D14,A06,sc,270,0,14,-0.481440,...,-0.069393,-0.234787,-4.932120,-1.852053,1.232599,1.310763,1.594189,0.075149,-0.793935,0.026775


In [16]:
# Extract out the cell painting features selected in this batch
batch_3_features = [x for x in cp_df.columns if not x.startswith("Metadata_")]

In [17]:
# Output combined file
file = os.path.join("data", "{}_combined_normalized_variable_selected.tsv".format(batch_id))
cp_df.to_csv(file, index=False, sep='\t')

In [18]:
replicate_cols = ["Metadata_Plate",
                  "Metadata_cell_line",
                  "Metadata_patient",
                  "Metadata_FFA",
                  "Metadata_diff_day"]

cp_collapsed_df = cp_df.groupby(replicate_cols).mean().reset_index()

## Use `write_gct.R` to build the Moxrpheus Input

In [19]:
%%R -i cp_df -i batch_id -i cp_df -i cp_collapsed_df

library(dplyr)
library(magrittr)

file <- file.path("..", "cytominer_scripts", "write_gct.R")
source(file)

output <- file.path("results", "morpheus",
                    paste0("full_", batch_id, "_morpheus.gct"))
output_collapsed <- file.path("results", "morpheus",
                              paste0("collapsed_", batch_id, "_morpheus.gct"))

channels <- NULL
create_row_annotations <- TRUE
feature_regex <- "^Nuclei_|^Cells_|^Cytoplasm_"

# Step 1: Output combined gct file
write_gct(x = cp_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

# Step 2: Output replicate collapsed gct file
write_gct(x = cp_collapsed_df,
          path = output_collapsed,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

## Combine Batch 1 and Batch 3 Profiles

In [20]:
combined_batch_features = list(set(batch_1_features + batch_3_features))
len(combined_batch_features)

688

In [21]:
batch_id = "2019_04_16_Batch1"
backend_dir = os.path.join("..", "..", "backend", batch_id)

plate_dirs = [os.path.join(backend_dir, x) for x in os.listdir(backend_dir)]

In [22]:
# Build full cell painting dataset
batch_one_df = load_files(plate_dirs, file_grep="normalized.csv").assign(Metadata_Batch="batch_one")
batch_one_df.Metadata_diff_day = batch_one_df.Metadata_diff_day.astype(str)

meta_cols = [x for x in batch_one_df.columns if x.startswith("Metadata")]

batch_one_df = batch_one_df.loc[:, meta_cols + combined_batch_features]

print(batch_one_df.shape)
batch_one_df.head(2)

reading ../../backend/2019_04_16_Batch1/BR00101080/BR00101080_normalized.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101081/BR00101081_normalized.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101075/BR00101075_normalized.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101079/BR00101079_normalized.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101077/BR00101077_normalized.csv with profile count: 56
reading ../../backend/2019_04_16_Batch1/BR00101083/BR00101083_normalized.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101082/BR00101082_normalized.csv with profile count: 60
reading ../../backend/2019_04_16_Batch1/BR00101076/BR00101076_normalized.csv with profile count: 54
reading ../../backend/2019_04_16_Batch1/BR00101078/BR00101078_normalized.csv with profile count: 60
(530, 698)


,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_cell_line,Metadata_patient,Metadata_FFA,Metadata_diff_day,Metadata_Batch,...,Nuclei_Correlation_K_DNA_Mito,Cells_Texture_Correlation_AGP_5_00,Nuclei_Granularity_13_DNA,Nuclei_Intensity_LowerQuartileIntensity_ER,Cytoplasm_Intensity_MassDisplacement_Mito,Cytoplasm_Texture_InfoMeas1_AGP_5_02,Cells_Correlation_K_AGP_DNA,Cells_Mean_SmallBODIPYObjects_Granularity_12_ER,Cytoplasm_RadialDistribution_RadialCV_AGP_2of4,Nuclei_AreaShape_MinFeretDiameter
0,BR00101080,B02,BR00101080,BR00101080,B02,vc,PAC_261,0,3,batch_one,...,2.535870,1.426350,-0.528130,-0.249057,1.797418,-0.605471,-1.408554,0.706541,-0.310373,3.088816
1,BR00101080,B03,BR00101080,BR00101080,B03,sc,PAC_246,0,3,batch_one,...,0.173398,-1.321442,0.278191,-1.551050,-1.414092,0.745403,-0.725325,-0.675685,-0.737790,0.134623


In [23]:
batch_id = "2019_08_06_Batch3"
backend_dir = os.path.join("..", "..", "backend", batch_id)

plate_dirs = [os.path.join(backend_dir, x) for x in os.listdir(backend_dir)]

In [24]:
batch_three_df = load_files(plate_dirs, file_grep="normalized.csv").assign(Metadata_Batch="batch_three")
batch_three_df.Metadata_diff_day = batch_three_df.Metadata_diff_day.astype(str)
batch_three_df.Metadata_patient = batch_three_df.Metadata_patient.astype(str)

meta_cols = [x for x in batch_three_df.columns if x.startswith("Metadata")]

batch_three_df = batch_three_df.loc[:, meta_cols + combined_batch_features]

print(batch_three_df.shape)
batch_three_df.head(2)

reading ../../backend/2019_08_06_Batch3/D14/D14_normalized.csv with profile count: 80
reading ../../backend/2019_08_06_Batch3/Day3/Day3_normalized.csv with profile count: 80
reading ../../backend/2019_08_06_Batch3/Day0/Day0_normalized.csv with profile count: 80
reading ../../backend/2019_08_06_Batch3/Day8/Day8_normalized.csv with profile count: 80
(320, 698)


,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_cell_line,Metadata_patient,Metadata_FFA,Metadata_diff_day,Metadata_Batch,...,Nuclei_Correlation_K_DNA_Mito,Cells_Texture_Correlation_AGP_5_00,Nuclei_Granularity_13_DNA,Nuclei_Intensity_LowerQuartileIntensity_ER,Cytoplasm_Intensity_MassDisplacement_Mito,Cytoplasm_Texture_InfoMeas1_AGP_5_02,Cells_Correlation_K_AGP_DNA,Cells_Mean_SmallBODIPYObjects_Granularity_12_ER,Cytoplasm_RadialDistribution_RadialCV_AGP_2of4,Nuclei_AreaShape_MinFeretDiameter
0,D14,A01,D14,D14,A01,sc,252,0,14,batch_three,...,3.152095,2.521861,0.653076,2.961831,1.587563,-2.171387,0.354149,1.181547,1.126526,-0.027749
1,D14,A02,D14,D14,A02,vc,252,0,14,batch_three,...,0.993630,1.646357,-0.969481,1.298121,2.357069,-0.066702,-0.219782,-0.032986,0.448699,-0.017106


In [25]:
combined_df = pd.concat([batch_one_df, batch_three_df], axis="rows").dropna(axis="columns")
combined_df.Metadata_patient = combined_df.Metadata_patient.astype(str)

print(combined_df.shape)
combined_df.head(2)

(850, 689)


,Metadata_Plate,Metadata_Well,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_well_position,Metadata_cell_line,Metadata_patient,Metadata_FFA,Metadata_diff_day,Metadata_Batch,...,Nuclei_Correlation_K_DNA_Mito,Cells_Texture_Correlation_AGP_5_00,Nuclei_Granularity_13_DNA,Nuclei_Intensity_LowerQuartileIntensity_ER,Cytoplasm_Intensity_MassDisplacement_Mito,Cytoplasm_Texture_InfoMeas1_AGP_5_02,Cells_Correlation_K_AGP_DNA,Cells_Mean_SmallBODIPYObjects_Granularity_12_ER,Cytoplasm_RadialDistribution_RadialCV_AGP_2of4,Nuclei_AreaShape_MinFeretDiameter
0,BR00101080,B02,BR00101080,BR00101080,B02,vc,PAC_261,0,3,batch_one,...,2.535870,1.426350,-0.528130,-0.249057,1.797418,-0.605471,-1.408554,0.706541,-0.310373,3.088816
1,BR00101080,B03,BR00101080,BR00101080,B03,sc,PAC_246,0,3,batch_one,...,0.173398,-1.321442,0.278191,-1.551050,-1.414092,0.745403,-0.725325,-0.675685,-0.737790,0.134623


In [26]:
# Recode ER features to BODIPY
combined_df.columns = [x.replace("_ER", "_BODIPY") if "_ER" in x else x for x in combined_df.columns]

In [27]:
# Output combined file
file = os.path.join("data", "batch1_batch3_combined_normalized_variable_selected.tsv")
combined_df.to_csv(file, index=False, sep='\t')

## Setup Data Specifically for Morpheus

In [28]:
# Load labels
file = os.path.join("data", "category_labels.csv")
label_df = pd.read_csv(file)
label_df.columns = ["Metadata_{}".format(x) for x in label_df.columns]

label_df

,Metadata_T2D_bin,Metadata_quantile_T2D,Metadata_percentile_T2D,Metadata_rank_T2D,Metadata_percentile_HOMA-IR,Metadata_rank_HOMA-IR,Metadata_IID,Metadata_category
0,0,NaN,87.50,161.0,47.83,88,m164,1
1,1,NaN,81.52,150.0,80.43,148,m246,1
2,1,NaN,23.91,44.0,39.67,73,m252,0
3,0,0.0,38.32,70.5,93.48,172,m266,1
4,1,0.0,57.61,106.0,8.70,16,m270,0
5,1,NaN,92.39,170.0,79.35,146,m272,1
6,0,NaN,93.48,172.0,16.85,31,m276,0


In [29]:
combined_df.loc[:, "Metadata_patient"] = ["m{}".format(x.strip("PAC_")) for x in combined_df.Metadata_patient]

# Merge labels
combined_df = label_df.merge(combined_df, left_on="Metadata_IID", right_on="Metadata_patient", how="inner")

print(combined_df.shape)
combined_df.head()

(580, 697)


,Metadata_T2D_bin,Metadata_quantile_T2D,Metadata_percentile_T2D,Metadata_rank_T2D,Metadata_percentile_HOMA-IR,Metadata_rank_HOMA-IR,Metadata_IID,Metadata_category,Metadata_Plate,Metadata_Well,...,Nuclei_Correlation_K_DNA_Mito,Cells_Texture_Correlation_AGP_5_00,Nuclei_Granularity_13_DNA,Nuclei_Intensity_LowerQuartileIntensity_BODIPY,Cytoplasm_Intensity_MassDisplacement_Mito,Cytoplasm_Texture_InfoMeas1_AGP_5_02,Cells_Correlation_K_AGP_DNA,Cells_Mean_SmallBODIPYObjects_Granularity_12_BODIPY,Cytoplasm_RadialDistribution_RadialCV_AGP_2of4,Nuclei_AreaShape_MinFeretDiameter
0,0,NaN,87.5,161.0,47.83,88,m164,1,BR00101080,B07,...,0.371120,-1.297283,1.105599,-1.802039,-1.487107,1.082764,-0.833071,0.578199,-1.124198,-0.039384
1,0,NaN,87.5,161.0,47.83,88,m164,1,BR00101080,B08,...,0.304797,0.484505,-0.502055,0.054194,0.237532,0.728483,0.413726,0.143368,-1.284908,-0.914331
2,0,NaN,87.5,161.0,47.83,88,m164,1,BR00101080,C07,...,-0.025049,-1.191517,1.167962,-1.361841,-1.382735,1.201711,0.326149,0.433415,-0.961971,-0.263225
3,0,NaN,87.5,161.0,47.83,88,m164,1,BR00101080,C08,...,0.976670,0.082851,-0.223535,-0.531298,0.669022,0.155175,-0.986018,1.655345,-0.190161,-0.216617
4,0,NaN,87.5,161.0,47.83,88,m164,1,BR00101080,D07,...,0.167839,-1.162118,1.243441,-1.551653,-1.285450,0.963578,-0.603963,0.007797,-0.244942,-0.009311


In [30]:
combined_df = combined_df.query("Metadata_FFA == 0")

In [31]:
replicate_cols = [
    "Metadata_Batch",
    "Metadata_Plate",
    "Metadata_cell_line",
    "Metadata_patient",
    "Metadata_FFA",
    "Metadata_diff_day"
]

combined_collapsed_df = combined_df.groupby(replicate_cols).mean().reset_index()

print(combined_collapsed_df.shape)
combined_collapsed_df.head(2)

(86, 692)


,Metadata_Batch,Metadata_Plate,Metadata_cell_line,Metadata_patient,Metadata_FFA,Metadata_diff_day,Metadata_T2D_bin,Metadata_quantile_T2D,Metadata_percentile_T2D,Metadata_rank_T2D,...,Nuclei_Correlation_K_DNA_Mito,Cells_Texture_Correlation_AGP_5_00,Nuclei_Granularity_13_DNA,Nuclei_Intensity_LowerQuartileIntensity_BODIPY,Cytoplasm_Intensity_MassDisplacement_Mito,Cytoplasm_Texture_InfoMeas1_AGP_5_02,Cells_Correlation_K_AGP_DNA,Cells_Mean_SmallBODIPYObjects_Granularity_12_BODIPY,Cytoplasm_RadialDistribution_RadialCV_AGP_2of4,Nuclei_AreaShape_MinFeretDiameter
0,batch_one,BR00101075,sc,m164,0,15,0,NaN,87.50,161.0,...,-0.794630,0.805323,-0.060673,3.630649,-0.727349,-0.220593,6.068930,1.014585,0.004909,-2.056625
1,batch_one,BR00101075,sc,m246,0,15,1,NaN,81.52,150.0,...,-0.692188,0.733186,0.721603,3.345600,-0.981709,-0.749595,5.515613,0.847641,-0.590615,-1.831379


## Use `write_gct.R` to output files for Morpheus

In [32]:
%%R -i combined_df -i combined_collapsed_df

library(dplyr)
library(magrittr)

file <- file.path("..", "cytominer_scripts", "write_gct.R")
source(file)

output <- file.path("results", "morpheus",
                    paste0("full_combined_batch1_batch3_morpheus.gct"))
output_collapsed <- file.path("results", "morpheus",
                              paste0("collapsed_full_combined_batch1_batch3_morpheus.gct"))

channels <- NULL
create_row_annotations <- TRUE
feature_regex <- "^Nuclei_|^Cells_|^Cytoplasm_"

# Step 1: Output combined gct file
write_gct(x = combined_df,
          path = output,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)

# Step 2: Output replicate collapsed gct file
write_gct(x = combined_collapsed_df,
          path = output_collapsed,
          channels = channels,
          create_row_annotations = create_row_annotations,
          feature_regex = feature_regex)